In [ ]:
api_key='RGAPI-473c5960-3ed2-4d30-822d-3c0861908f09'
import requests
import time
import pandas as pd

# Retrieving high rank solo queue ladder

In [ ]:
#get chal, gm, master leaderboards and convert to one
root= 'https://euw1.api.riotgames.com/'
challenger_root = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
grandmaster_root = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
master_root = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'

challenger_response = requests.get(root+challenger_root+'?api_key='+api_key)
grandmaster_response = requests.get(root+grandmaster_root+'?api_key='+api_key)
master_response = requests.get(root+master_root+'?api_key='+api_key)

challenger_df = pd.DataFrame(challenger_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
grandmaster_df=pd.DataFrame(grandmaster_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
master_df=pd.DataFrame(master_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)


#concatenating the dataframes
high_rank_ladder=pd.concat([challenger_df, grandmaster_df, master_df])[0:1500].reset_index(drop=True)
high_rank_ladder=high_rank_ladder.drop(columns='rank').reset_index(drop=False).rename(columns={'index':'rank'})
high_rank_ladder['rank']+=1

high_rank_ladder

In [ ]:
def get_puuid_from_summoner_id(summonerId=None):
    '''takes summoner id as parameter and gets puuid'''
    link=f'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}?api_key={api_key}'

    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('rate limit exceeded')
            time.sleep(10)
            continue
        puuid=response.json()['puuid']
        return puuid

In [ ]:
puuid_list = []
#gets puuids for every high rank player
for summoner in high_rank_ladder['summonerId']:
    puuids = get_puuid_from_summoner_id(summonerId=summoner)
    puuid_list.append(puuids)

In [ ]:
def get_match_id_from_puuid(puuid=None):
    """
    takes puuid as parameter and gets most recent 15 ranked matches
    """
    link=f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&type=ranked&start=0&count=15&api_key={api_key}'
    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('rate limit exceeded')
            time.sleep(10)
            continue
        match_id_list=response.json()
        break
    return match_id_list


In [ ]:
all_match_id_list=[]
#gets match ids for every puuid
for player_puuid in puuid_list:
    match_ids=get_match_id_from_puuid(puuid=player_puuid)
    all_match_id_list.extend(match_ids)

In [ ]:
len(all_match_id_list)

In [ ]:
from collections import Counter

#count duplicates and remove them from match id list
match_id_counts = Counter(all_match_id_list)

duplicate_match_ids = [match_id for match_id, count in match_id_counts.items() if count > 1]

unique_match_id_list = list(set(all_match_id_list))

In [ ]:
len(unique_match_id_list)

In [ ]:
def get_match_info_at_twenty(matchId=None):
    '''takes match id as parameter and gets the match json file'''
    link = link= f'https://europe.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}'
    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('Rate limit exceeded')
            time.sleep(10)
            continue
        elif response.status_code==200:
            break
        elif response.status_code==404:
            return 0
        else:
            print('Response is bad: '+str(response.status_code))
    match_data = response.json()

    if match_data is None:
        print('no data obtained')
        return 0

    #checks if the match is a ranked match
    if match_data['info']['queueId'] ==420:
        timeline_api_link = f'https://europe.api.riotgames.com/lol/match/v5/matches/{matchId}/timeline?api_key={api_key}'

        while True:
            timeline_response = requests.get(timeline_api_link)
            if timeline_response.status_code==429:
                print('Rate limit exceeded')
                time.sleep(10)
                continue
            elif timeline_response.status_code==200:
                break
            elif timeline_response.status_code==404:
                print('timeline info not found')
                return 0
            else:
                print('Response is bad: '+str(timeline_response.status_code))
        timeline_info = timeline_response.json()
        return timeline_info
    
    else:
        print('not a ranked match')
        #return 0
            


In [ ]:
get_match_info_at_twenty(matchId='EUW1_6920157556')

In [ ]:
import pandas as pd

In [ ]:
def get_specific_match_stats_at_twenty(data):

    '''takes match json as parameter (data) and processes it to get info we are interested in'''
    blue_side = data['info']['participants'][:5]
    red_side = data['info']['participants'][5:]

    #blue side statistics
    #we will not consider baron in this because it spawns at exactly 20 minutes so it's impossible for any team to kill it in time
    total_blue_side_minions = 0
    total_blue_side_kills=0
    total_blue_side_deaths=0
    total_blue_side_gold_earned = 0
    total_blue_side_wards_placed = 0
    total_towers_destroyed_by_blue = 0
    total_inhibitors_destroyed_by_blue =0
    total_blue_side_dragons_killed =0
    total_blue_side_riftHeralds_killed =0
    blue_team_first_blood =0
    blue_side_won=0

    #red side statistics
    total_red_side_minions = 0
    total_red_side_kills=0
    total_red_side_deaths =0
    total_red_side_gold_earned = 0
    total_red_side_wards_placed = 0
    total_towers_destroyed_by_red = 0
    total_inhibitors_destroyed_by_red =0
    total_red_side_dragons_killed =0
    total_red_side_riftHeralds_killed =0
 



    #getting if blue side won or not (target variable, 1= blue, 0 = red)    
    for event in data['info']['frames'][-1]['events']:
        if event['type'] == 'GAME_END':
            if event['winningTeam'] ==100:
                blue_side_won+=1
    
    for participant in blue_side:
        total_blue_side_minions += data['info']['frames'][19]['participantFrames'][str(participant['participantId'])]['minionsKilled']
        total_blue_side_gold_earned+= data['info']['frames'][19]['participantFrames'][str(participant['participantId'])]['totalGold']

    for participant in red_side:
        total_red_side_minions += data['info']['frames'][19]['participantFrames'][str(participant['participantId'])]['minionsKilled']
        total_red_side_gold_earned+= data['info']['frames'][19]['participantFrames'][str(participant['participantId'])]['totalGold']


    #for loop to iterate over the events that occurred at the first 20 minutes/'frames' in the game
    for frame in range(20):
        for event in data['info']['frames'][frame]['events']:
            if event['type']=='WARD_PLACED':
                if event['creatorId'] in range(1,6):
                    total_blue_side_wards_placed+=1
                else:
                    total_red_side_wards_placed+=1

            #kill stats for both teams
            if event['type']=='CHAMPION_KILL':
                if event['killerId'] in range(1,6):
                    total_blue_side_kills+=1
                    total_red_side_deaths+=1
                else:
                    total_red_side_kills+=1
                    total_blue_side_deaths+=1
            
            #objective stats for both teams
            if event['type']=='BUILDING_KILL' and event['buildingType'] == 'TOWER_BUILDING':
                if event['killerId'] in range (1,6):
                    total_towers_destroyed_by_blue+=1
                else:
                    total_towers_destroyed_by_red+=1
            
            #tower/turret stats
            if event['type']=='BUILDING_KILL' and event['buildingType'] == 'INHIBITOR_BUILDING':
                if event['killerId'] in range (1,6):
                    total_inhibitors_destroyed_by_blue+=1
                else:
                    total_inhibitors_destroyed_by_red+=1

            #elite monsters stats. within first 20 mins the only elite monsters that can be killed are rift heralds and dragons, baron nashor and elder dragon spawns after 20 mmins
            if event['type']=='ELITE_MONSTER_KILL':
                if event['monsterType'] == 'DRAGON':
                    if event['killerId'] in range(1,6):
                        total_blue_side_dragons_killed+=1
                    else:
                        total_red_side_dragons_killed+=1
                if event['monsterType']=='RIFTHERALD':
                    if event['killerId'] in range(1,6):
                        total_blue_side_riftHeralds_killed+=1
                    else:
                        total_red_side_riftHeralds_killed+=1
                    


    df = pd.DataFrame({
        #blue stats
        'total_blue_side_minions': [total_blue_side_minions],
        'total_blue_side_gold_earned': [total_blue_side_gold_earned],
        'total_blue_side_wards_placed':[total_blue_side_wards_placed],
        'total_blue_side_kills':[total_blue_side_kills],
        'total_blue_side_deaths':[total_blue_side_deaths],
        'total_blue_side_dragons_killed':[total_blue_side_dragons_killed],
        'total_blue_side_riftHeralds_killed':[total_blue_side_riftHeralds_killed],
        'total_towers_destroyed_by_blue': [total_towers_destroyed_by_blue],
        'total_inhibitors_destroyed_by_blue':[total_inhibitors_destroyed_by_blue],

        #red stats
        'total_red_side_minions':[total_red_side_minions],
        'total_red_side_gold_earned':[total_red_side_gold_earned],
        'total_red_side_kills':[total_red_side_kills],
        'total_red_side_deaths':[total_red_side_deaths],
        'total_red_side_wards_placed':[total_red_side_wards_placed],        
        'total_red_side_dragons_killed':[total_red_side_dragons_killed],
        'total_red_side_riftHeralds_killed':[total_red_side_riftHeralds_killed],
        'total_towers_destroyed_by_red':[total_towers_destroyed_by_red],
        'total_inhibitors_destroyed_by_red':[total_inhibitors_destroyed_by_red],

        'blue_side_won': [blue_side_won]


    })

    return df
        



In [ ]:
len(all_match_id_list)

In [ ]:
match_ids = unique_match_id_list
dfs = pd.DataFrame()  #empty dataframe to hold all our data
#gets all match info for match ids
for match_id in match_ids:
    try:
        game = get_match_info_at_twenty(matchId=match_id)

        if game is None:
            print(f'{match_id} is none. skip')
            continue
        #check if match is actually 20 minutes long
        if len(game['info']['frames']) >=20:  
            match_df = get_specific_match_stats_at_twenty(game)
            dfs=pd.concat([dfs, match_df])
        else:
            print(f"Match ID {match_id} is shorter than 20 minutes.")
    except TypeError as e:
        print(f'error with {match_id}: {e}')

In [ ]:
dfs.shape

In [ ]:
dfs

In [ ]:
dfs.to_csv('twenty_mins_data.csv')

In [ ]:
df =pd.read_csv('twenty_mins_data.csv')
df